# **MIP2_Run1**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """You are the production planner in a food and beverage production fascility. You are tasked with planning production for the next 12 time periods with the goal of maximizing profit. The production fascility is able to produce two types of food and three types of beverage. The food is produced in whole units, while the beverage can be produced in any fraction of a liter. For each product that is produced at period t, a fixed cost and variable cost incurred. Each product can also be stored which is also associated with a variable cost per product stored. There is also a maximum demand for each product and time period which means that a specific product will not be sold anymore if the demand is met for that time period. Please note, that the demand does not have to be met. The revenue per product and time also differs. The cost for production and storage, the demand and revenue are all provided as in sepearte csv files named "fixed_cost_production.csv", "variable_cost_production.csv", "variable_cost_storage.csv", "demand.csv" and "revenue.csv" where all files follow the structre of the columns representing time and rows representing product. Finally, there is a constant storage capacity and production capacity for each product that resets each time period. The storage capacity is 580 units of food 1, 687 units of food 2, 599L of beverage one, 788 Liters of beverage 2 and 294L of beverage 3. The production capacity is 1080 units of food 1, 908 units of food 2, 408L of beverage one, 1000L of beverage 2 and 403L of beverage 3. """

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [6]:
print(response1.choices[0].message.content)

1. Decision Variables:
   - Let X[i,t] represent the amount of product i to be produced in time period t
   - Let S[i,t] represent the amount of product i to be stored in time period t

2. Parameters:
   - Let F[i,t] represent the fixed cost of production of product i in time period t from "fixed_cost_production.csv"
   - Let V[i,t] represent the variable cost of production of product i in time period t from "variable_cost_production.csv"
   - Let VS[i,t] represent the variable cost of storage of product i in time period t from "variable_cost_storage.csv"
   - Let D[i,t] represent the demand of product i in time period t from "demand.csv"
   - Let R[i,t] represent the revenue per unit product i in time period t from "revenue.csv"
   - Let SC[i] represent the storage capacity of product i
   - Let PC[i] represent the production capacity of product i

3. Constraints parameters:
   - SC = [580, 687, 599, 788, 294], the maximum storage capacity for each product
   - PC = [1080, 908, 408, 1

In [9]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [10]:
print(response2.choices[0].message.content)

The objective function to maximize the total profit gained over all the time periods and for all the products can be represented as:

Max ∑∑((R[i,t]*min(D[i,t],X[i,t])) - (F[i,t] + V[i,t]*X[i,t] + VS[i,t]*S[i,t])) for all i and t.

This equation aims to maximize the profit calculated as the revenue gained from selling the products (minimum of demand and production for each product) minus the total costs which includes the fixed cost of production, variable cost of production, and variable cost of storage. 

The sum is performed over all the time periods and across all the products.


In [11]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [12]:
print(response3.choices[0].message.content)

1. Non-negativity constraints:
   - For all products i in all time periods t, X[i,t] >= 0
   - For all products i in all time periods t, S[i,t] >= 0

2. Production capacity constraint:
   - For all products i in all time periods t, X[i,t] <= PC[i]. This guarantees that we never produce more than our total production capacity.

3. Storage capacity constraint:
   - For all products i in all time periods t, S[i,t] <= SC[i]. This guarantees that we never store more than our total storage capacity.

4. Production and Demand Constraint: 
   - For products i in all time periods t, X[i,t] <= D[i,t]. This guarantees not to produce more than the demand.

5. Balance Constraint (Quantity produced + Quantity Stored from previous period = Quantity Sold + Quantity Stored):
   - For all products i at time period 1, X[i,1] = min(D[i,1],X[i,1]) + S[i,1]
   - For all products i and time periods t incremented from 2 to 12, X[i,t] + S[i,t-1] = min(D[i,t],X[i,t]) + S[i,t]

This ensures that the quantity pro

### **Generate Pyomo Code**

In [18]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [19]:
Markdown(response4.choices[0].message.content)

Here is the sample python pyomo code for this optimization problem:

```python
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Indexes
model.I = Set(initialize=[1,2,3,4,5]) # 5 products
model.T = Set(initialize=[1,2,3,4,5,6,7,8,9,10,11,12]) # 12 time periods

#Sample Parameters: (Actual Data would come from the csv files named in the problem statement)
F = {(i,t): 50 for i in model.I for t in model.T} #Fixed costs (use pd.read_csv('fixed_cost_production.csv') to import csv file)
V = {(i,t): 10 for i in model.I for t in model.T} #Variable cost of production 
VS = {(i,t): 2 for i in model.I for t in model.T} #Variable cost of storage 
D = {(i,t): 100 for i in model.I for t in model.T} #Demand
R = {(i,t): 60 for i in model.I for t in model.T} #Revenue 
SC = {i: 580 for i in model.I} #Storage Capacity
PC = {i: 1080 for i in model.I} #Production Capacity

# Decision Variables
model.X = Var(model.I, model.T, domain=NonNegativeReals)
model.S = Var(model.I, model.T, domain=NonNegativeReals)

# Objective Function
model.profit = Objective(expr = sum((R[i,t]*min(D[i,t],model.X[i,t])) - (F[i,t] + V[i,t]*model.X[i,t] + VS[i,t]*model.S[i,t]) for i in model.I for t in model.T), sense=maximize)

# Constraints
model.prod_capacity = Constraint(model.I, model.T, rule = lambda model, i, t: model.X[i,t] <= PC[i])
model.store_capacity = Constraint(model.I, model.T, rule = lambda model, i, t: model.S[i,t] <= SC[i])
model.prod_demand = Constraint(model.I, model.T, rule = lambda model, i, t: model.X[i,t] <= D[i,t])

# Balance Constraints
def balance_rule(model, i, t):
    if t == 1:
        return model.X[i,1] == min(D[i,1],model.X[i,1]) + model.S[i,1]
    if t > 1:
        return model.X[i,t] + model.S[i,t-1] == min(D[i,t],model.X[i,t]) + model.S[i,t]

model.balance = Constraint(model.I, model.T, rule=balance_rule)

# instantiate and solve the solver 
sol = SolverFactory('glpk').solve(model)

```
Please modify the F,V,VS,D,R,SC,PC parameters according to your actual data files and needs. The code assumes a certain structure for your data sets. Please modify the code accordingly if your data sets are structured differently.

### **Run the code Generated by GPT4**

In [20]:
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Indexes
model.I = Set(initialize=[1,2,3,4,5]) # 5 products
model.T = Set(initialize=[1,2,3,4,5,6,7,8,9,10,11,12]) # 12 time periods

# Sample Parameters: (Actual Data would come from the csv files named in the problem statement)
F = {(i,t): 50 for i in model.I for t in model.T} #Fixed costs (use pd.read_csv('fixed_cost_production.csv') to import csv file)
V = {(i,t): 10 for i in model.I for t in model.T} #Variable cost of production
VS = {(i,t): 2 for i in model.I for t in model.T} #Variable cost of storage
D = {(i,t): 100 for i in model.I for t in model.T} #Demand
R = {(i,t): 60 for i in model.I for t in model.T} #Revenue
SC = {i: 580 for i in model.I} #Storage Capacity
PC = {i: 1080 for i in model.I} #Production Capacity

# Decision Variables
model.X = Var(model.I, model.T, domain=NonNegativeReals)
model.S = Var(model.I, model.T, domain=NonNegativeReals)

# Objective Function
model.profit = Objective(expr = sum((R[i,t]*min(D[i,t],model.X[i,t])) - (F[i,t] + V[i,t]*model.X[i,t] + VS[i,t]*model.S[i,t]) for i in model.I for t in model.T), sense=maximize)

# Constraints
model.prod_capacity = Constraint(model.I, model.T, rule = lambda model, i, t: model.X[i,t] <= PC[i])
model.store_capacity = Constraint(model.I, model.T, rule = lambda model, i, t: model.S[i,t] <= SC[i])
model.prod_demand = Constraint(model.I, model.T, rule = lambda model, i, t: model.X[i,t] <= D[i,t])

# Balance Constraints
def balance_rule(model, i, t):
    if t == 1:
        return model.X[i,1] == min(D[i,1],model.X[i,1]) + model.S[i,1]
    if t > 1:
        return model.X[i,t] + model.S[i,t-1] == min(D[i,t],model.X[i,t]) + model.S[i,t]

model.balance = Constraint(model.I, model.T, rule=balance_rule)

# instantiate and solve the solver
sol = SolverFactory('glpk').solve(model)

PyomoException: Cannot convert non-constant Pyomo expression (X[1,1]  <  100) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**